In [40]:
import requests
import time
from bs4 import BeautifulSoup
from requests.exceptions import RequestException

MAX_REQUESTS = 5
REQUEST_DELAY_SECONDS = 60
MIN_RESULTS = 3

def request_info(url):
    for i in range(MAX_REQUESTS):
        response = requests.get(url)
        if response.status_code == 429:
            time.sleep(REQUEST_DELAY_SECONDS)
        else:
            break
    if response.status_code != 200:
        print(f"Error requesting {url}: status code {response.status_code}")
        return None
    return response

def get_food_urls(food_name, page=0, verbose=False):
    search_base_url = "https://www.fatsecret.es/calorías-nutrición/search?q="
    page_url = "&pg=" + str(page)
    url = search_base_url.strip() + food_name.strip().replace(" ", "+") + page_url
    if verbose:
        print(f"Searching on {url}")
    response = request_info(url)
    if response is None:
        return None
    soup = BeautifulSoup(response.text, "html.parser")
    food_links = soup.find_all("td", {"class": "borderBottom"})
    results = [(link.find("a", {"class": "prominent"}).text, "https://fatsecret.es" + link.find("a", {"class": "prominent"})["href"]) for link in food_links]
    return tuple(results)


def check_100g(food_tuple, verbose=False):
    if verbose:
        print(f"Checking if {food_tuple[0]} is 100g -- {food_tuple[1]}")
    soup = BeautifulSoup(request_info(food_tuple[1]).text, "html.parser")
    serving_size = soup.find("div", {"class": "serving_size black serving_size_value"})
    return serving_size is not None and serving_size.text == "100 g"


def get_macros(food_tuple, verbose=False):
    if verbose:
        print(f"Gathering macros for {food_tuple[0]} -- {food_tuple[1]}")
    soup = BeautifulSoup(request_info(food_tuple[1]).text, "html.parser")
    macros = soup.find("div", {"class": "factPanel"})
    if macros is None:
        print(f"No macros found for {food_tuple[0]}")
        return None
    macro_list = [macro.text for macro in macros.find_all("div", {"class": "factValue"})]
    if len(macro_list) < 4:
        print(f"Not enough macros found for {food_tuple[0]}")
        return None
    cals, fat, carb, prot = [float(macro.replace("g", "").replace(",", ".")) for macro in macro_list[:4]]
    return {"cals": cals, "fat": fat, "carb": carb, "prot": prot}

def get_metadata(food_tuple, verbose=False):

    if verbose:
        print(f"Gathering metadata for {food_tuple[0]} -- {food_tuple[1]}")
    url = food_tuple[1]
    try:
        response = request_info(url)
    except RequestException as e:
        print(f"Error requesting {url}: {e}")
        return None
    
    if response.status_code != 200:
        print(f"Error requesting {url}: status code {response.status_code}")
        return None
    
    soup = BeautifulSoup(response.text, "html.parser")
    
    # BRAND
    brand = None
    brand_tag = soup.find("a", {"style": "color: #028CC4;"})
    if brand_tag is not None:
        brand = brand_tag.text

    return {"brand": brand}

def get_food_info(food_name, min_results=MIN_RESULTS, verbose=False):
    data = []
    tuples = []
    page = 0
    while len(tuples) < min_results:
        if verbose:
            print(f"Searching for {food_name} on page {page}")
        new_tuples = get_food_urls(food_name, page, verbose)
        tuples += [t for t in new_tuples if check_100g(t, verbose)]
        if verbose:
            print(f"Tuples for {food_name} found: {len(tuples)}")
        page += 1
    for t in tuples:
        macros = get_macros(t, verbose)
        if macros is not None:
            metadata = get_metadata(t, verbose)
            data.append({**macros, **metadata})
    return data

def get_food_info_list(food_list, min_results=MIN_RESULTS, verbose=True):
    return [food_info for food in food_list for food_info in get_food_info(food, min_results, verbose)]
food_list = ["garrapiñada "]

data = get_food_info_list(food_list, verbose=True)


Searching for garrapiñada on page 0
Searching on https://www.fatsecret.es/calorías-nutrición/search?q=garrapiñada&pg=0
Checking if Garrapiñadas de Almendras is 100g -- https://fatsecret.es/calor%C3%ADas-nutrici%C3%B3n/gen%C3%A9rico/garrapi%C3%B1adas-de-almendras
Checking if Cacahuete Garrapiñado is 100g -- https://fatsecret.es/calor%C3%ADas-nutrici%C3%B3n/hacendado/cacahuete-garrapi%C3%B1ado/100g
Checking if Almendra Garrapiñada is 100g -- https://fatsecret.es/calor%C3%ADas-nutrici%C3%B3n/hacendado/almendra-garrapi%C3%B1ada/100g
Checking if Cacahuete Garrapiñado Con Miel is 100g -- https://fatsecret.es/calor%C3%ADas-nutrici%C3%B3n/carrefour/cacahuete-garrapi%C3%B1ado-con-miel/100g
Checking if Garrapiñada de Maní is 100g -- https://fatsecret.es/calor%C3%ADas-nutrici%C3%B3n/gen%C3%A9rico/garrapi%C3%B1ada-de-man%C3%AD
Tuples for garrapiñada found: 5
Gathering macros for Garrapiñadas de Almendras -- https://fatsecret.es/calor%C3%ADas-nutrici%C3%B3n/gen%C3%A9rico/garrapi%C3%B1adas-de-almend

In [41]:
import json
with open('data.json', 'w') as outfile:
    json.dump(data, outfile)